<a href="https://colab.research.google.com/github/FireDragon00/LaTeX-Template-Cn/blob/master/%E8%AE%BA%E6%96%87%E5%AE%9E%E9%AA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pandas
!pip install torch
!pip install dgl

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from sklearn import model_selection
from IPython.display import display, HTML
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# 初始化参数

In [4]:
root_path = '/content/drive/MyDrive/researchHub/MAppGraph/mappgraph/data'

In [5]:
N = 20
t = 10
k = 10  # the number of rows for the output tensor (k = 10, 20)
T = 5
overlap = 3 # note: overlap depends on T

apps = ['diijam', 'baomoi', 'fptplay', 'iQIYI', 'bigo', 'myradio', 'spotify', 'nhaccuatui', 'soundcloud', 'sachnoiapp', 
        'phim247', 'popskid', 'truyenaudiosachnoiviet', 'vieon', 'voizfm', 'tunefm', 'wetv', 'zingmp3', 'truyenaudio', 'baohay24h',
        'freefire', 'among_us', 'azar', 'comico', 'nimotv', 'mangatoon', 'medoctruyen', 'nhacvang', 'noveltoon', 'radiofm',
        'vtvgo', 'tivi24h', 'tinder', 'tinmoi24h', 'tivi360', 'tiktok', 'linkedin', 'tiki', 'tinhte', 'lotus', 'tivi247',
        'tivi_truyentranh_webtoon', 'tuoitre_online', 'vietnamworks', 'wallstreet_journal', 'cnn_news', 'bbc_news', 'twitter', 
        'weeboo', 'twitch', 'vnexpress', 'topcv', 'toc_chien', 'wesing', 'hago', 'google_meet', 'dubsmash', 'facebook','hahalolo', 
        'zalo', 'hello_yo', 'dan_tri', 'zoom', 'wikipedia', 'instagram', 'jobway', 'kaka', 'pinterest', 'quora', 'lazada', 'chess', 
        'cake', 'mobile_legend', 'co_tuong_online', 'ted', 'telegram', 'starmarker', 'skype', 'soha', 'tango', 'thanhnien', 'snapchat', 
        'tien_len', 'animal_restaurant', 'bida', 'cho_tot', 'messenger', 'netflix', 'nonolive', 'may', 'podcast_player', 'pubg', 
        'partying', 'kenh14', 'lienquan_mobile', 'likee_lite', 'reddit', 'sendo', 'shopee', 'the_guardian', 'ola_party']

features = ['complete_max', 'complete_min', 'complete_mean', 'complete_mad', 'complete_std', 'complete_var', 'complete_skew',
       'complete_kurt', 'complete_pkt_num', 'complete_10per', 'complete_20per', 'complete_30per', 'complete_40per', 'complete_50per', 
        'complete_60per', 'complete_70per', 'complete_80per', 'complete_90per', 'out_max', 'out_min', 'out_mean', 'out_mad', 'out_std',
        'out_var', 'out_skew', 'out_kurt', 'out_pkt_num', 'out_10per', 'out_20per', 'out_30per', 'out_40per', 'out_50per', 'out_60per',
        'out_70per', 'out_80per', 'out_90per', 'in_max', 'in_min', 'in_mean', 'in_mad', 'in_std', 'in_var', 'in_skew', 'in_kurt', 
        'in_pkt_num', 'in_10per', 'in_20per', 'in_30per', 'in_40per', 'in_50per', 'in_60per', 'in_70per', 'in_80per', 'in_90per', 
        'protocol', 'flows_num', 'flow_length_mean', 'flow_pkt_num_mean', 'flow_duration_mean', 'ip1', 'ip2', 'ip3', 'ip4'
       ]

In [6]:
train_graphs_folder = os.path.join(root_path, '%d_%d/train_graphs/N%d/t%d'%(T, overlap, N, t))
test_graphs_folder = os.path.join(root_path, '%d_%d/test_graphs/N%d/t%d'%(T, overlap, N, t))

# 从一个app文件夹中加载图

In [7]:
import torch
import dgl

'''
Load graphs for one app
Input: app and folder that contains graphs of the app
Output: List of graphs (StellarGraph objects) and List of labels
'''


def graphs_one_app(app, graphs_folder):
	graphs = []

	app_graph_folder = os.path.join(graphs_folder, app)
	features_path = os.path.join(app_graph_folder, 'features.csv')
	weights_path = os.path.join(app_graph_folder, 'weights.csv')

	features_df = pd.read_csv(features_path, index_col=0)
	weights_df = pd.read_csv(weights_path, index_col=0)

	graph_num = features_df['graph_id'].iloc[-1]
	# loop over all graphs of the app
	for i in range(1, graph_num + 1):
		feature_df = features_df[features_df['graph_id'] == i]
		feature_df = feature_df[['IP_port'] + features + ['graph_id']]
		weight_df = weights_df[weights_df['graph_id'] == i]

		# drop graph_id column
		feature_df = feature_df.drop(['graph_id'], axis=1)
		weight_df = weight_df.drop(['graph_id'], axis=1)
	
		# ip_port to node_id
		ip_port = list(pd.concat([weight_df['source'], weight_df['target']]).value_counts().index)
		node_id = [i for i in range(len(ip_port))]

		weight_df = weight_df.replace(ip_port,node_id)
		feature_df = feature_df.replace(ip_port, node_id).sort_values(by=['IP_port']).set_index('IP_port')
	
		# .reset_index(drop=True)


		if weight_df.shape[0] > 0:
			
			u, v = torch.tensor(pd.concat([weight_df['source'],weight_df['target']]).values.astype('int32')),\
			torch.tensor(pd.concat([weight_df['target'],weight_df['source']]).values.astype('int32'))
			# print(u)
			g = dgl.graph((u, v))
	 
			weights = torch.tensor(pd.concat([weight_df['weight'],weight_df['weight']]).values.astype('float32'))
			g.edata['w'] = weights

			feats = torch.tensor(np.array(feature_df.values).astype('float32'))
			g.ndata['attr'] = feats
			graphs.append(g)
			# print(g)

	return graphs

Using backend: pytorch


# 从所有APP中加载图

In [8]:
'''
Load all graphs
Input: folder that contains graphs
Output: List of graphs (StellarGraph objects), List of graph_labels (dummy values) and List of labels (names of app)
'''


def generate_graphs(graphs_folder):
	# build a list of graphs and labels: note that only apply for more than 2 classes
	li = []
	labels = []
	idx = 0

	for app in apps:
		
		print('Loading {} ... {}/{}'.format(app, idx, len(apps)))

		one_app_graphs = graphs_one_app(app, graphs_folder)
		# print(len(one_app_graphs))
		one_app_labels = [idx for _ in range(len(one_app_graphs))]
		# print(len(one_app_labels))
		li.extend(one_app_graphs)
		labels.extend(one_app_labels)
		idx += 1

	graph_labels = labels
	graphs = li

	print('...............................................................')

	return graphs, graph_labels

In [ ]:
train_graphs, train_graph_labels = generate_graphs(train_graphs_folder)
test_graphs, test_graph_labels = generate_graphs(test_graphs_folder)
train_size = len(train_graphs)

In [11]:
print(len(train_graph_labels))
print(len(test_graph_labels))

66236
16561


In [32]:
from dgl.data.utils import save_graphs

train_labels = {"labels": torch.tensor(train_graph_labels)}
test_labels = {"labels": torch.tensor(test_graph_labels)}

save_graphs("/content/drive/MyDrive/train_graphs.bin", train_graphs, train_labels)
save_graphs("/content/drive/MyDrive/test_graphs.bin", test_graphs, test_labels)
# graphs, label = load_graphs('sample/train_graphs.bin')
# print(type(graphs), type(label))
# print(graphs[0])

In [30]:
from dgl.data import DGLDataset
from dgl.data.utils import load_graphs

# g, l = load_graphs('sample/train_graphs.bin')

class Traindataset(DGLDataset):
  _url = 'http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/qm7b.mat'
  _sha1_str = '4102c744bb9d6fd7b40ac67a300e49cd87e28392'

  def __init__(self, raw_dir=None, force_reload=False, verbose=False):
    super().__init__(name='qm7b',
            url=self._url,
            raw_dir=raw_dir,
            force_reload=force_reload,
            verbose=verbose)

  def process(self):
    # 将数据处理为图列表和标签列表
    self.graphs, label_dict = load_graphs('sample/train_graphs.bin')
    self.label = label_dict['train_labels']
    # self.graphs, self.label = train_graphs, train_graph_labels

  def __getitem__(self, idx):
    """ 通过idx获取对应的图和标签

    Parameters
    ----------
    idx : int
        Item index

    Returns
    -------
    (dgl.DGLGraph, Tensor)
    """
    return self.graphs[idx], self.label[idx]

  def __len__(self):
    """数据集中图的数量"""
    return len(self.graphs)
  
  @property
  def num_labels(self):
    return 101

In [31]:
from dgl.dataloading import GraphDataLoader

dataset = Traindataset()
dataloader = GraphDataLoader(
  dataset,
  batch_size=1024,
  drop_last=False,
  shuffle=True)
print(dataset[0])

(Graph(num_nodes=3, num_edges=6,
      ndata_schemes={'attr': Scheme(shape=(63,), dtype=torch.float32)}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)}), tensor(0))


In [33]:
import dgl
import torch
import dgl.nn.pytorch as dglnn
import torch.nn as nn
from dgl.nn.pytorch.conv import GATConv
from dgl.nn.pytorch.conv import DenseGraphConv
from dgl.nn.pytorch.glob import SortPooling
import torch.nn.functional as F


class Classifier(nn.Module):
  def __init__(self, in_dim, hidden_dim, n_classes, batch_size, num_graph):
    super(Classifier, self).__init__()
    self.batch_size = batch_size
    self.num_graph = num_graph
    self.gat1 = GATConv(in_dim, hidden_dim, 5)
    self.gat2 = GATConv(hidden_dim*5, hidden_dim, 5)
    
    #l2_out: torch.Size([3371, 5, 20])==>l2_flatten: torch.Size([3371, 100])
    self.sortpooling = SortPooling(k=16)
    #sortpool: torch.Size([1024, 1600])
    self.conv1D_1 = nn.Conv1d(1, 128, kernel_size=3)
    self.maxpooling = nn.MaxPool1d(3)
    self.conv1D_2 = nn.Conv1d(128, 256, kernel_size=2)
    self.classify = nn.Linear(256*531, n_classes)

  def forward(self, g, h):
    # print('raw_shape:',h.size())
    # GATConv
    h = F.relu(self.gat1(g, h))
    # print('l1_out:', h.size())
    h = h.flatten(1)
    # print('l1_flatten:', h.size())
    h = F.relu(self.gat2(g, h))
    # print('l2_out:', h.size())
    h = h.flatten(1)
    # print('l2_flatten:', h.size())

    # sortpool——图池化
    h = self.sortpooling(g, h)
    # print('pool1:', h.size())
    try:
      h = h.view(1024, 1, 1600)
    except:
      x = self.num_graph % self.batch_size
      h = h.view(x , 1, 1600)
    # print('view:', h.size())
    h = self.conv1D_1(h)
    # print('conv1:', h.size())
    h = self.maxpooling(h)
    # print('pool2:', h.size())
    h = self.conv1D_2(h)
    # print('conv2:', h.size())
    h = F.relu(h.flatten(1))
    h = F.dropout(h)
    with g.local_scope():
        return self.classify(h)

In [ ]:
model = Classifier(63, 20, 101, 1024, 66236)
opt = torch.optim.Adam(model.parameters(), lr=0.001)
model.train()

epoch_losses = []
for epoch in range(50):
  epoch_loss = 0
  i = 0
  for batched_graph, labels in dataloader:
      feats = batched_graph.ndata['attr']
      logits = model(batched_graph, feats)
      # print(logits.size())
      loss = F.cross_entropy(logits, labels)
      opt.zero_grad()
      loss.backward()
      opt.step()
      epoch_loss += loss.detach().item()
      i += 1
  epoch_loss /= (i+1)
  print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss))
  epoch_losses.append(epoch_loss)